In [3]:
import numpy as np

# dados
X = np.arange(start=0, stop=9).reshape(3, 3)
y = np.array([[1.], [0.], [1.]])

# funções de ativação
def relu(z):
    return np.maximum(0, z)
def sigmoid(z):
    z_ = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z_))
def binary_cross_entropy(y_pred, y_true):
    return -(y_true / y_pred) + ((1 - y_true) / (1 - y_pred))
        
# camadas (cada linha é um neuronio)
theta1 = np.ones((3, 3))
theta2 = np.ones((1, 3))

# forward
z1 = np.dot(X, theta1.T)
a1 = relu(z1)

z2 = np.dot(a1, theta2.T)
a2 = sigmoid(z2)

In [ ]:
class Tensor:
    def __init__(self, data):
        # array numpy
        self.data = np.array(data, dtype=np.float16)
        
        # gradiente
        self.grad = np.zeros_like(self.data, dtype=np.float32)
        
        # função que calcula o gradiente local
        self.local_grad = lambda: None 
        
        # guarda os tensores que que participaram da operação
        self._prev = set()
    
    
    def dot(self, other):
        '''
        Faz a operação de produto interno entre dois tensores.
        Args:
            other (Tensor): Tensor que irá entrar no produto interno.
        '''
        
        # converte "other" para tensor caso não seja
        other = other if isinstance(other, Tensor) else Tensor(other)
        
        # resultado da operação
        out = Tensor(np.dot(self.data, other.data))
        
        def _backward():
            # gradiente em relação ao self
            self.grad += np.dot(out.grad, other.data.T)
            
            # gradiente em relação ao other
            other.grad += np.dot(self.data.T, out.grad)
        
        # atualiza os gradientes dos tensores
        out.local_grad = _backward
        out._prev = {self, other}
        
        return out
    
    def backward(self):
        # armazena a ordem dos tensores
        graph = []
        visited = set()
        
        # função que constrói o grafo
        def build_graph(tensor):
            if tensor not in visited:
                visited.add(tensor)
                for child in tensor._prev:
                    build_graph(child)
                graph.append(tensor)
        
        build_graph(self)
        
        # inicializa o gradiente do tensor final (loss) como 1
        self.grad = np.ones_like(self.data)
        
        # propaga o gradiente na ordem inversa
        for tensor in reversed(graph):
            tensor._backward()
        
        
    def __relu__(self):
        '''Faz a ativação ReLu.'''
        
        out = Tensor(np.maximum(0, self.data))
        
        def _backward():
            self.grad += (self.data > 0) * out.grad
        
        out.local_grad = _backward
        out._prev = {self}
        
        return out


    def __sigmoid__(self):
        '''Faz a ativação Sigmoid.'''
        
        z = np.clip(self.data, -500, 500)
        sigmoid_ = 1 / (1 + np.exp(-z))
        out = Tensor(sigmoid_)
        
        def _backward():
            self.grad += sigmoid_ * (1 - sigmoid_) * out.grad 
            
        out.local_grad = _backward
        out._prev = {self}
        
        return out
    

    def __sub__(self, other):
        if isinstance(other, Tensor):
            return Tensor(self.data - other.data)
        else:
            return Tensor(self.data - other)
    def __truediv__(self, other):
        if isinstance(other, Tensor):
            return Tensor(self.data / other.data)
        else:
            return Tensor(self.data / other)
    def __mul__(self, other):
        if isinstance(other, Tensor):
            return Tensor(self.data * other.data)
        else:
            return Tensor(self.data * other)
        
    def __add__(self, other):
        """
        Implementa a operação de soma entre dois Tensores ou um Tensor e um número.
        """
        # Converte "other" para Tensor caso não seja
        other = other if isinstance(other, Tensor) else Tensor(other)
        
        # Resultado da soma
        out = Tensor(self.data + other.data)
        
        def _backward():
            # O gradiente da soma é simplesmente repassado para os operandos
            self.grad += out.grad
            other.grad += out.grad
        
        # Define o método backward e registra os predecessores
        out.local_grad = _backward
        out._prev = {self, other}
        
        return out

        
    @property
    def T(self):
        '''Permite fazer a transposição da matriz de dados direto no objeto.'''
        return Tensor(self.data.T)
    
    @property
    def shape(self):
        '''Permite acessar o shape da matriz de dados direto do objeto.'''
        return self.data.shape
    
    def __repr__(self):
        ''' Permite imprimir a matriz numpy ao chamar o objeto.'''
        return f'{self.data}'



def relu(tensor):
    return tensor.__relu__()

def sigmoid(tensor):
    return tensor.__sigmoid__()

def mse_loss(pred, target):
    """
    Calcula o erro quadrático médio entre a previsão e o alvo.
    Args:
        pred (Tensor): Valores previstos.
        target (Tensor): Valores reais.
    Returns:
        Tensor: Valor escalar representando o erro.
    """
    # Converte target para Tensor se necessário
    target = target if isinstance(target, Tensor) else Tensor(target)
    
    # (pred - target) ** 2
    diff = pred - target
    loss = diff * diff / len(target.data)
    
    def _backward():
        # Derivada do MSE em relação a pred: 2 * (pred - target) / n
        pred.grad += 2 * diff.data / len(target.data) * loss.grad
        # Derivada do MSE em relação a target é oposta a pred
        target.grad -= 2 * diff.data / len(target.data) * loss.grad
    
    loss._backward = _backward
    loss._prev = {pred, target}
    
    return loss

In [ ]:
class Tensor:
    def __init__(self, data):
        # array numpy
        self.data = np.array(data, dtype=np.float16)
        
        # gradiente
        self.grad = np.zeros_like(self.data, dtype=np.float32)
        
        # função que calcula o gradiente local
        self.local_grad = lambda: None 
        
        # guarda os tensores que que participaram da operação
        self._prev = set()
    
    def dot(self, other):
        '''
        Faz a operação de produto interno entre dois tensores.
        Args:
            other (Tensor): Tensor que irá entrar no produto interno.
        '''
        
        # converte "other" para tensor caso não seja
        other = other if isinstance(other, Tensor) else Tensor(other)
        
        # resultado da operação
        out = Tensor(np.dot(self.data, other.data))
        
        def _backward():
            # gradiente em relação ao self
            self.grad += np.dot(out.grad, other.data.T)
            
            # gradiente em relação ao other
            other.grad += np.dot(self.data.T, out.grad)
        
        # atualiza os gradientes dos tensores
        out.local_grad = _backward
        out._prev = {self, other}
        
        return out
    

    def __sub__(self, other):
        if isinstance(other, Tensor):
            return Tensor(self.data - other.data)
        else:
            return Tensor(self.data - other)
    def __truediv__(self, other):
        if isinstance(other, Tensor):
            return Tensor(self.data / other.data)
        else:
            return Tensor(self.data / other)
    def __mul__(self, other):
        if isinstance(other, Tensor):
            return Tensor(self.data * other.data)
        else:
            return Tensor(self.data * other)
        
    def __add__(self, other):
        """
        Implementa a operação de soma entre dois Tensores ou um Tensor e um número.
        """
        # Converte "other" para Tensor caso não seja
        other = other if isinstance(other, Tensor) else Tensor(other)
        
        # Resultado da soma
        out = Tensor(self.data + other.data)
        
        def _backward():
            # O gradiente da soma é simplesmente repassado para os operandos
            self.grad += out.grad
            other.grad += out.grad
        
        # Define o método backward e registra os predecessores
        out.local_grad = _backward
        out._prev = {self, other}
        

    def __repr__(self):
        ''' Permite imprimir a matriz numpy ao chamar o objeto.'''
        return f'Tensor({self.data})'

In [154]:
class Tensor:
    def __init__(self, data, chain=None, parents=None, operation=None):
        self.data = np.array(data, dtype=np.float32)
        self.grad = np.zeros_like(self.data)
        self.operation = operation
        self.parents = parents or []
        self.chain = chain
    
    def __add__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        return Tensor(self.data + other.data, parents=[self, other], operation='add')
    def __mul__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        
        out = Tensor(self.data * other.data)
        
        def _backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
            
        out.operation = 'mul'
        out.parents = [self, other]
        out.chain = _backward
        
        return out
    
    def __repr__(self):
        return f'Tensor({self.data}, operation={self.operation})'
    


x = Tensor(np.array([3]))
w = Tensor(np.array([2]))
b = Tensor(np.array([1]))

y = x * w

# Quase certo

In [274]:
import numpy as np

class Tensor:
    def __init__(self, data, parents=None, operation=None):
        self.data = np.array(data, dtype=np.float32)  # Assegura que os dados sejam tratados como matrizes
        self.grad = np.zeros_like(self.data, dtype=np.float32)  # Inicializa o gradiente com a mesma forma
        self.parents = parents or []  # Lista de pais da operação
        self.operation = operation  # Tipo de operação (se houver)
        
    def backward(self, grad=1.0):
        self.grad += grad
        print(self)
        print()
        for parent, local_grad in self.parents:
            parent.backward(grad *local_grad(self))
    
    # Função para calcular o erro quadrático médio (MSE)
    @staticmethod
    def mse_loss(y_pred, y_true):
        loss = ((y_pred.data - y_true.data) ** 2).mean()
        
        return Tensor(
            loss, 
            parents=[
                (y_pred, lambda _: 2 * (y_pred.data - y_true.data) / y_pred.data.size), 
                (y_true, lambda _: -2 * (y_pred.data - y_true.data) / y_true.data.size)],
            operation='mse'
        )

    def __add__(self, other):
        '''Permite fazer a operação de adição entre Tensores.'''
        
        other = other if isinstance(other, Tensor) else Tensor(other)
        return Tensor(
            self.data + other.data, 
            parents=[(self, lambda _: 1), (other, lambda _: 1)], 
            operation='add'
        )
    
    def __mul__(self, other):
        '''Permite fazer a operação de Multiplicação de Hadamar (elemento com elemento) entre Tensores.'''
        
        other = other if isinstance(other, Tensor) else Tensor(other)
        return Tensor(
            self.data * other.data,
            parents=[(self, lambda _: other.data), (other, lambda _: self.data)],
            operation='mul'
        )
    
    def __sub__(self, other):
        '''Permite fazer a operação de subtração entre Tensores'''
        
        other = other if isinstance(other, Tensor) else Tensor(other)
        return Tensor(
            self.data - other.data,
            parents=[(self, lambda _: 1), (other, lambda _: 1)],
            operation='sub'
        )
    
    def __truediv__(self, other):
        '''Permite fazer a operação de divisão entre Tensores.'''
        
        other = other if isinstance(other, Tensor) else Tensor(other)
        return Tensor(
            self.data / other.data,
            parents=[(self, lambda _: 1/other.data), (other, lambda _: -self.data/other.data**2)],
            operation='div'
        )
        
    def __matmul__(self, other):
        '''Permite fazer o produto interno entre dois Tensores.'''
        
        other = other if isinstance(other, Tensor) else Tensor(other)
        return Tensor(
            np.matmul(self.data, other.data),
            parents=[(self, lambda _: other.data), (other, lambda _: self.data)],
            operation='matmul'
        )
        
    def __repr__(self):
        '''Permite visualizar alguns parâmetros do Tensor.'''
        
        return f'Tensor({str(self.data)}, operation={self.operation})'
    
    @property
    def T(self):
        '''Permite fazer a transposição da matriz de dados direto no objeto.'''
        
        return Tensor(self.data.T)
    
    @property
    def shape(self):
        '''Permite acessar o shape da matriz de dados direto do objeto.'''
        
        return self.data.shape


X = np.array([[1, 2, 3], [4, 5, 6]])
y = np.array([[10], [15]])

w = np.array([[1, 1, 1]])
b = np.array([[1], [1]])

X = Tensor(X)
w = Tensor(w)
b = Tensor(b)

pred = X @ w.T + b
loss = Tensor.mse_loss(pred, y)
loss.backward()


Tensor(5.0, operation=mse)

Tensor([[ 7.]
 [16.]], operation=add)

Tensor([[ 6.]
 [15.]], operation=matmul)



ValueError: operands could not be broadcast together with shapes (2,1) (3,1) 

Tensor(5.0, operation=mse)

Tensor([[ 7.]
 [16.]], operation=add)

Tensor([[ 6.]
 [15.]], operation=matmul)



ValueError: operands could not be broadcast together with shapes (2,1) (3,1) 

In [241]:
(X @ w.T).grad

array([[0.],
       [0.]], dtype=float32)

In [237]:
X @ w.T

Tensor([[ 6.]
 [15.]], operation=matmul)

In [ ]:
def backward(node: Tensor):
    """
    Propaga os gradientes para trás através do grafo computacional.
    
    Args:
        node (Tensor): O nó a partir do qual o backward deve começar.
    """
    # Primeiro, exibe o nó atual e o gradiente
    print(f"Backward passando pelo nó: {node}")
    
    # Para cada nó, aplicamos a operação de diferenciação e propagação do gradiente
    if node.parents:
        for parent in node.parents:
            # Cada operação tem uma forma diferente de calcular o gradiente.
            # Aqui, podemos aplicar a regra da cadeia para cada tipo de operação.

            # Exemplo de regra para a operação de multiplicação (mul)
            if node.operation == "mul":
                parent.grad += node.grad * parent.data  # gradiente da multiplicação
            # Exemplo de regra para a operação de soma (add)
            elif node.operation == "add":
                parent.grad += node.grad  # gradiente da soma (simples)

            # Recursivamente propaga para os pais
            backward(parent)


In [ ]:
y.parents

[Tensor([6.], operation=mul), Tensor([1.], operation=None)]

In [99]:
h = 0.01
(max(0, y.data + h) - max(0, y.data)) / h

array([1.172], dtype=float16)

In [81]:
import torch 

x = torch.arange(4.0, requires_grad=True)

y = torch.dot(x, x)

y.backward()

In [82]:
x, x.grad

(tensor([0., 1., 2., 3.], requires_grad=True), tensor([0., 2., 4., 6.]))

In [35]:
X = np.arange(start=0, stop=9).reshape(3, 3)
y = np.array([[1.], [0.], [1.]])

X = Tensor(X)
y = Tensor(y)

In [36]:
tensor1 = Tensor(theta1)
tensor2 = Tensor(theta2)

for i in range(3):
    z1 = X.dot(tensor1.T)
    a1 = relu(z1)

    z2 = a1.dot(tensor2.T)
    a2 = sigmoid(z2)

    loss = mse_loss(pred=a2, target=y)

    loss.backward()

    print(loss.data.sum())

0.3333
0.3333
0.3333


In [39]:
tensor1.grad

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [30]:
loss.data.sum()

16.84

In [6]:
for i in range(5):
    z1 = X.dot(tensor1.T)
    a1 = relu(z1)

    z2 = a1.dot(tensor2.T)
    a2 = sigmoid(z2)

    loss = ((a2 - y) * (a2 - y)).sum()
    loss.backward()

AttributeError: 'Tensor' object has no attribute 'sum'

In [ ]:
a2 * y

[[1.]
 [0.]
 [1.]]

In [ ]:
a2

[[1.]
 [1.]
 [1.]]

In [ ]:
loss = ((a2 - y) * (a2 - y)).sum()

1.0

In [ ]:
-(y * Tensor(np.log(a2.data)) + Tensor(1 - y) * Tensor(np.log(1 - a2.data)))

TypeError: unsupported operand type(s) for -: 'int' and 'Tensor'

In [ ]:
y * Tensor(np.log(a2.data))

[[-0.00012339]
 [ 0.        ]
 [ 0.        ]]

In [ ]:
y * Tensor(np.log(a2.data))

TypeError: unsupported operand type(s) for *: 'Tensor' and 'Tensor'

In [ ]:
y

[[1.]
 [0.]
 [1.]]